## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_Vacation_Search import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df = city_data_df.set_index("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
City_ID,,,,,,,,,
0,Hamilton,US,39.18,-84.53,70.0,77,90,9.17,2020-09-11 23:55:41
1,Port Hedland,AU,-20.32,118.57,75.2,60,0,5.82,2020-09-11 23:55:41
2,Punta Arenas,CL,-53.15,-70.92,42.8,70,90,13.87,2020-09-11 23:52:30
3,Kahului,US,20.89,-156.47,91.4,43,40,24.16,2020-09-11 23:52:01
4,Conde,BR,-7.26,-34.91,75.2,78,40,8.05,2020-09-11 23:55:42


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = input("What is the Maximum Temperature you would like for your trip (Fahrenheit): ")
min_temp = input("What is the Minimum Temperature you would like for your trip (Fahrenheit): ")

What is the Maximum Temperature you would like for your trip (Fahrenheit): 60
What is the Minimum Temperature you would like for your trip (Fahrenheit): 56


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= int(max_temp)) & (city_data_df["Max Temp"] >= int(min_temp))]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Date          0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Cloudiness", "Humidity", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Cloudiness,Humidity,Lat,Lng,Hotel Name
City_ID,,,,,,,,
28,Svetlaya,RU,59.81,0,63,46.54,138.33,
50,Mount Gambier,AU,57.20,90,93,-37.83,140.77,
58,Zhangye,CN,59.79,89,41,38.93,100.45,
73,Warrington,GB,57.20,75,93,53.39,-2.58,
105,Chuy,UY,56.26,59,91,-33.70,-53.46,
114,Carnarvon,AU,59.00,0,67,-24.87,113.63,
128,Moron,AR,59.00,75,38,-34.65,-58.62,
159,Saint Anthony,US,57.20,90,71,45.02,-93.22,
161,Lasa,IT,57.20,75,93,46.62,10.70,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"]=f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Nearby API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    city_data_hotel_data = requests.get(url, params=params).json()
    #city_data_hotel_df
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = city_data_hotel_data["results"][0]["name"]
    except:
        print("City not Found.")

City not Found.
City not Found.
City not Found.
City not Found.
City not Found.
City not Found.
City not Found.
City not Found.
City not Found.
City not Found.


In [23]:
hotel_df.head()

,City,Country,Max Temp,Cloudiness,Humidity,Lat,Lng,Hotel Name
City_ID,,,,,,,,
28,Svetlaya,RU,59.81,0,63,46.54,138.33,
50,Mount Gambier,AU,57.20,90,93,-37.83,140.77,Commodore on the Park
58,Zhangye,CN,59.79,89,41,38.93,100.45,Zhangye Lvzhou Holiday Inn
73,Warrington,GB,57.20,75,93,53.39,-2.58,Villaggio Hotel
105,Chuy,UY,56.26,59,91,-33.70,-53.46,Nuevo Hotel Plaza


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].dropna(axis=0, how="any")

City_ID
28                                                      
50                                 Commodore on the Park
58                            Zhangye Lvzhou Holiday Inn
73                                       Villaggio Hotel
105                                    Nuevo Hotel Plaza
114                                Hospitality Carnarvon
128                                          Hotel Morón
159               Holiday Inn Express Roseville-St. Paul
161                                Pension Feldgärtenhof
170                                       Westmark Sitka
176                                                     
195                                 Le Relais Saint Jean
202                           Hotel Atlântico Rio Grande
234                                                     
247                                           Hotel Got7
270                           Chenango Valley State Park
299                               Hotel complex Pastoral
314                    

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "C:\\Users\\amira\\OneDrive\\Documents\\Data Science\\Repositories\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))